In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
# Open the schools data file
schools = os.path.join("raw_data", "schools_complete.csv")
schools_df = pd.read_csv(schools)
# schools_df.head()

In [3]:
# Open the students data file
students = os.path.join("raw_data", "students_complete.csv")
students_df = pd.read_csv(students)
# students_df.head()

In [4]:
# schools_df.dtypes

In [5]:
# students_df.dtypes

In [6]:
## Part One - District Summary

# Key district metrics for distric summary table

total_schools  = len(schools_df)

total_students = schools_df["size"].sum()

total_budget = schools_df["budget"].sum()

avg_math_score = students_df["math_score"].mean()

avg_reading_score = students_df["reading_score"].mean()

# % Passing Math based on 70
math_pass = students_df.loc[(students_df["math_score"] >= 70)]
math_count = math_pass["math_score"].count()

# Need to pull in total students by school
passing_math = math_count/total_students*100

# % Passing Math based on 70
read_pass = students_df.loc[(students_df["reading_score"] >= 70)]
read_count = read_pass["reading_score"].count()
passing_reading = read_count/total_students*100

overall_passing = (passing_math + passing_reading)/2

In [7]:
# Create district summary table
district_summary_table = pd.DataFrame({"Total Schools": [total_schools],
                                      "Total Students": [total_students],
                                      "Total Budget": [total_budget,],
                                      "Average Math Score": [avg_math_score],
                                      "Average Reading Score": [avg_reading_score],
                                      "% Passing Math": [passing_math],
                                      "% Passing Reading": [passing_reading],
                                      "Overall Passing Score": [overall_passing],
                                      })
district_summary_table = district_summary_table[["Total Schools",
                                                 "Total Students",
                                                 "Total Budget",
                                                 "Average Math Score",
                                                 "Average Reading Score",
                                                 "% Passing Math",
                                                 "% Passing Reading",
                                                 "Overall Passing Score",
                                                 ]]
district_summary_table = district_summary_table.round(2)

# Format table
district_summary_table["Total Students"] = district_summary_table["Total Students"].map("{0:,.0f}".format)
district_summary_table["Total Budget"] = district_summary_table["Total Budget"].map("{0:,.0f}".format)
district_summary_table["% Passing Math"] = district_summary_table["% Passing Math"].map("{0:,.2f}%".format)
district_summary_table["% Passing Reading"] = district_summary_table["% Passing Reading"].map("{0:,.2f}%".format)
district_summary_table["Overall Passing Score"] = district_summary_table["Overall Passing Score"].map("{0:,.2f}%".format)

##Part One Answer
#Print Table
print("District Summary")
district_summary_table

District Summary


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Score
0,15,"39,170","24,649,428",78.99,81.88,74.98%,85.81%,80.39%


In [8]:
##Part Two - School Summary

# Add per student budget to schools_df
schools_df["Per Student Budget"] = schools_df["budget"]/schools_df["size"]

# Renames "name" to "school"
schools_df = schools_df.rename(columns={"name": "school"})

# Merge student and school tables
district_df = pd.merge(students_df, schools_df, on="school")
district_df.head()

,Student ID,name,gender,grade,school,reading_score,math_score,School ID,type,size,budget,Per Student Budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635,655.0
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635,655.0
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635,655.0
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635,655.0
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635,655.0


In [9]:
# Get the mean test scores by school and merge into schools_df
average_scores_table = district_df.groupby(["school"])['reading_score', 'math_score'].mean().reset_index()

schools_df = schools_df.merge(average_scores_table, on = 'school', how = "outer")

In [10]:
## Get passing counts so we can calculate the percentage
# Get math counts
student_math_pass = district_df[district_df['math_score'] >=70]
math_pass_count = student_math_pass.groupby(["school"])['math_score'].count().reset_index()
math_pass_count.rename_axis({'math_score' : 'Math Count'}, axis=1, inplace=True)

# Get reading counts
student_reading_pass = district_df[district_df['reading_score'] >=70]
reading_pass_count = student_reading_pass.groupby(["school"])['reading_score'].count().reset_index()
reading_pass_count.rename_axis({'reading_score' : 'Reading Count'}, axis=1, inplace=True)

#Merge counts into one table
pass_count = math_pass_count.merge(reading_pass_count, on="school", how='inner')
pass_count

# Now merge into schools_df for calucalations
schools_df = schools_df.merge(pass_count, on = 'school', how = "outer")

schools_df

,School ID,school,type,size,budget,Per Student Budget,reading_score,math_score,Math Count,Reading Count
0,0,Huang High School,District,2917,1910635,655.0,81.182722,76.629414,1916,2372
1,1,Figueroa High School,District,2949,1884411,639.0,81.158020,76.711767,1946,2381
2,2,Shelton High School,Charter,1761,1056600,600.0,83.725724,83.359455,1653,1688
3,3,Hernandez High School,District,4635,3022020,652.0,80.934412,77.289752,3094,3748
4,4,Griffin High School,Charter,1468,917500,625.0,83.816757,83.351499,1371,1426
5,5,Wilson High School,Charter,2283,1319574,578.0,83.989488,83.274201,2143,2204
6,6,Cabrera High School,Charter,1858,1081356,582.0,83.975780,83.061895,1749,1803
7,7,Bailey High School,District,4976,3124928,628.0,81.033963,77.048432,3318,4077
8,8,Holden High School,Charter,427,248087,581.0,83.814988,83.803279,395,411
9,9,Pena High School,Charter,962,585858,609.0,84.044699,83.839917,910,923


In [11]:
# Calculate the percent passing for math and reading, and then the overall passing
schools_df["% Passing Math"] = schools_df["Math Count"]/schools_df['size']*100
schools_df["% Passing Reading"] = schools_df["Reading Count"]/schools_df['size']*100
schools_df["% Overall Passing"] = (schools_df["% Passing Math"] + schools_df["% Passing Reading"])/2

# Delect math and reading counts no longer needed
del schools_df['Math Count']
del schools_df['Reading Count']
del schools_df['School ID']

# making a copy of schools_df before formatting it
copy_schools_df = schools_df.copy(deep=True)

In [12]:
# Format table
schools_df["size"] = schools_df["size"].map("{0:,.0f}".format)
schools_df["budget"] = schools_df["budget"].map("{0:,.0f}".format)
schools_df["Per Student Budget"] = schools_df["Per Student Budget"].map("{0:,.0f}".format)
schools_df["reading_score"] = schools_df["reading_score"].map("{0:,.2f}".format)
schools_df["math_score"] = schools_df["math_score"].map("{0:,.2f}".format)
schools_df["% Passing Math"] = schools_df["% Passing Math"].map("{0:,.2f}%".format)
schools_df["% Passing Reading"] = schools_df["% Passing Reading"].map("{0:,.2f}%".format)
schools_df["% Overall Passing"] = schools_df["% Overall Passing"].map("{0:,.2f}%".format)

print("School Summary")
schools_df

School Summary


,school,type,size,budget,Per Student Budget,reading_score,math_score,% Passing Math,% Passing Reading,% Overall Passing
0,Huang High School,District,"2,917","1,910,635",655,81.18,76.63,65.68%,81.32%,73.50%
1,Figueroa High School,District,"2,949","1,884,411",639,81.16,76.71,65.99%,80.74%,73.36%
2,Shelton High School,Charter,"1,761","1,056,600",600,83.73,83.36,93.87%,95.85%,94.86%
3,Hernandez High School,District,"4,635","3,022,020",652,80.93,77.29,66.75%,80.86%,73.81%
4,Griffin High School,Charter,"1,468","917,500",625,83.82,83.35,93.39%,97.14%,95.27%
5,Wilson High School,Charter,"2,283","1,319,574",578,83.99,83.27,93.87%,96.54%,95.20%
6,Cabrera High School,Charter,"1,858","1,081,356",582,83.98,83.06,94.13%,97.04%,95.59%
7,Bailey High School,District,"4,976","3,124,928",628,81.03,77.05,66.68%,81.93%,74.31%
8,Holden High School,Charter,427,"248,087",581,83.81,83.80,92.51%,96.25%,94.38%
9,Pena High School,Charter,962,"585,858",609,84.04,83.84,94.59%,95.95%,95.27%


In [13]:
sorted_schools = schools_df.sort_values("% Overall Passing", ascending=False)
print("Top Performing Schools")
sorted_schools.head()

Top Performing Schools


,school,type,size,budget,Per Student Budget,reading_score,math_score,% Passing Math,% Passing Reading,% Overall Passing
6,Cabrera High School,Charter,"1,858","1,081,356",582,83.98,83.06,94.13%,97.04%,95.59%
14,Thomas High School,Charter,"1,635","1,043,130",638,83.85,83.42,93.27%,97.31%,95.29%
4,Griffin High School,Charter,"1,468","917,500",625,83.82,83.35,93.39%,97.14%,95.27%
9,Pena High School,Charter,962,"585,858",609,84.04,83.84,94.59%,95.95%,95.27%
5,Wilson High School,Charter,"2,283","1,319,574",578,83.99,83.27,93.87%,96.54%,95.20%


In [14]:
print("Bottom Performing Schools")
sorted_schools.tail()

Bottom Performing Schools


,school,type,size,budget,Per Student Budget,reading_score,math_score,% Passing Math,% Passing Reading,% Overall Passing
13,Ford High School,District,"2,739","1,763,916",644,80.75,77.10,68.31%,79.30%,73.80%
12,Johnson High School,District,"4,761","3,094,650",650,80.97,77.07,66.06%,81.22%,73.64%
0,Huang High School,District,"2,917","1,910,635",655,81.18,76.63,65.68%,81.32%,73.50%
1,Figueroa High School,District,"2,949","1,884,411",639,81.16,76.71,65.99%,80.74%,73.36%
11,Rodriguez High School,District,"3,999","2,547,363",637,80.74,76.84,66.37%,80.22%,73.29%


In [15]:
##Part Five - Rading Score by Grade
math_scores_grade = pd.pivot_table(district_df, index = ["school"],
                                   values = ("math_score"), 
                                   columns = ["grade"], aggfunc=np.mean)

# Clean up header and reorder columns
math_flattened = pd.DataFrame(math_scores_grade.to_records())
math_flattened = math_flattened[["school", "9th", "10th", "11th", "12th"]]

print("Math Scores by Grade")
math_flattened

Math Scores by Grade


,school,9th,10th,11th,12th
0,Bailey High School,77.083676,76.996772,77.515588,76.492218
1,Cabrera High School,83.094697,83.154506,82.765560,83.277487
2,Figueroa High School,76.403037,76.539974,76.884344,77.151369
3,Ford High School,77.361345,77.672316,76.918058,76.179963
4,Griffin High School,82.044010,84.229064,83.842105,83.356164
5,Hernandez High School,77.438495,77.337408,77.136029,77.186567
6,Holden High School,83.787402,83.429825,85.000000,82.855422
7,Huang High School,77.027251,75.908735,76.446602,77.225641
8,Johnson High School,77.187857,76.691117,77.491653,76.863248
9,Pena High School,83.625455,83.372000,84.328125,84.121547


In [16]:
##Part Six - Rading Score by Grade
reading_scores_grade = pd.pivot_table(district_df, index = ["school"],
                                   values = ("reading_score"), 
                                   columns = ["grade"], aggfunc=np.mean)

# Clean up header and reorder columns
reading_flattened = pd.DataFrame(reading_scores_grade.to_records())
reading_flattened = reading_flattened[["school", "9th", "10th", "11th", "12th"]]

print("Reading Scores by Grade")
reading_flattened

Reading Scores by Grade


,school,9th,10th,11th,12th
0,Bailey High School,81.303155,80.907183,80.945643,80.912451
1,Cabrera High School,83.676136,84.253219,83.788382,84.287958
2,Figueroa High School,81.198598,81.408912,80.640339,81.384863
3,Ford High School,80.632653,81.262712,80.403642,80.662338
4,Griffin High School,83.369193,83.706897,84.288089,84.013699
5,Hernandez High School,80.866860,80.660147,81.396140,80.857143
6,Holden High School,83.677165,83.324561,83.815534,84.698795
7,Huang High School,81.290284,81.512386,81.417476,80.305983
8,Johnson High School,81.260714,80.773431,80.616027,81.227564
9,Pena High School,83.807273,83.612000,84.335938,84.591160


In [17]:
# Create scores by spending table
bins = [0,585,615,645,1000]
group_labels = ["0-585", "585 to 615", "615 to 645", "645+"]

# Making a copy of copy_schools_df for use in the size exercise
size_schools_df = copy_schools_df.copy(deep=True)
pd.cut(copy_schools_df["Per Student Budget"],bins,labels = group_labels).head()

0          645+
1    615 to 645
2    585 to 615
3          645+
4    615 to 645
Name: Per Student Budget, dtype: category
Categories (4, object): [0-585 < 585 to 615 < 615 to 645 < 645+]

In [18]:
copy_schools_df["Budget Group"] = pd.cut(copy_schools_df["Per Student Budget"],bins,labels=group_labels)
del copy_schools_df['size']
del copy_schools_df['budget']
del copy_schools_df['Per Student Budget']
copy_schools_df.head()

,school,type,reading_score,math_score,% Passing Math,% Passing Reading,% Overall Passing,Budget Group
0,Huang High School,District,81.182722,76.629414,65.683922,81.316421,73.500171,645+
1,Figueroa High School,District,81.158020,76.711767,65.988471,80.739234,73.363852,615 to 645
2,Shelton High School,Charter,83.725724,83.359455,93.867121,95.854628,94.860875,585 to 615
3,Hernandez High School,District,80.934412,77.289752,66.752967,80.862999,73.807983,645+
4,Griffin High School,Charter,83.816757,83.351499,93.392371,97.138965,95.265668,615 to 645


In [19]:
district_budget = copy_schools_df.groupby("Budget Group")
pd.DataFrame(district_budget.size().reset_index())
print("Scores by Spending")
district_budget.mean()

Scores by Spending


,reading_score,math_score,% Passing Math,% Passing Reading,% Overall Passing
Budget Group,,,,,
0-585,83.933814,83.455399,93.460096,96.610877,95.035486
585 to 615,83.885211,83.599686,94.230858,95.900287,95.065572
615 to 645,81.891436,79.079225,75.668212,86.106569,80.887391
645+,81.027843,76.997210,66.164813,81.133951,73.649382


In [20]:
# Scores by size
bins = [0,1000,2000,5000]
size_group_labels = ["<1000", "1000 to 2000", "2000 and up"]

# Making a copy of copy_schools_df for use in the size exercise
pd.cut(size_schools_df["size"],bins,labels = size_group_labels).head()

0     2000 and up
1     2000 and up
2    1000 to 2000
3     2000 and up
4    1000 to 2000
Name: size, dtype: category
Categories (3, object): [<1000 < 1000 to 2000 < 2000 and up]

In [21]:
size_schools_df["Size Group"] = pd.cut(size_schools_df["size"],bins,labels=size_group_labels)
del size_schools_df['budget']
del size_schools_df['Per Student Budget']
size_schools_df.head()

,school,type,size,reading_score,math_score,% Passing Math,% Passing Reading,% Overall Passing,Size Group
0,Huang High School,District,2917,81.182722,76.629414,65.683922,81.316421,73.500171,2000 and up
1,Figueroa High School,District,2949,81.158020,76.711767,65.988471,80.739234,73.363852,2000 and up
2,Shelton High School,Charter,1761,83.725724,83.359455,93.867121,95.854628,94.860875,1000 to 2000
3,Hernandez High School,District,4635,80.934412,77.289752,66.752967,80.862999,73.807983,2000 and up
4,Griffin High School,Charter,1468,83.816757,83.351499,93.392371,97.138965,95.265668,1000 to 2000


In [22]:
district_size = size_schools_df.groupby("Size Group")
pd.DataFrame(district_size.size().reset_index())
print("Scores by School Size")
district_size.mean()

Scores by School Size


,size,reading_score,math_score,% Passing Math,% Passing Reading,% Overall Passing
Size Group,,,,,,
<1000,694.500,83.929843,83.821598,93.550225,96.099437,94.824831
1000 to 2000,1704.400,83.864438,83.374684,93.599695,96.790680,95.195187
2000 and up,3657.375,81.344493,77.746417,69.963361,82.766634,76.364998


In [23]:
# Scores by type of school
del size_schools_df['Size Group']
by_type = size_schools_df.groupby("type")
pd.DataFrame(by_type.size().reset_index())
print("Scores by School Type")
by_type.mean()

Scores by School Type


,size,reading_score,math_score,% Passing Math,% Passing Reading,% Overall Passing
type,,,,,,
Charter,1524.250000,83.896421,83.473852,93.620830,96.586489,95.103660
District,3853.714286,80.966636,76.956733,66.548453,80.799062,73.673757
